In [6]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [7]:
shortforms = ['MHC']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [8]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [12]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-24 15:19:37] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [13]:
result = [grounding_map, names, pos_labels]

In [14]:
result

[{'histocompatibility complex molecules': 'MESH:D008285',
  'histocompatibility molecules': 'MESH:D008285',
  'major his tocompatibility complex': 'MESH:D008285',
  'major histo compatibility complex': 'MESH:D008285',
  'major histocom patibility complex': 'MESH:D008285',
  'major histocompability complex': 'MESH:D008285',
  'major histocompati bility complex': 'MESH:D008285',
  'major histocompatibil ity complex': 'MESH:D008285',
  'major histocompatibility': 'MESH:D008285',
  'major histocompatibility antigen': 'MESH:D008285',
  'major histocompatibility antigen complex': 'MESH:D008285',
  'major histocompatibility class': 'MESH:D008285',
  'major histocompatibility com plex': 'MESH:D008285',
  'major histocompatibility complex': 'MESH:D008285',
  'major histocompatibility complex antigen': 'MESH:D008285',
  'major histocompatibility complex class': 'MESH:D008285',
  'major histocompatibility complex proteins': 'MESH:D008285',
  'major histocompatibility ibility complex': 'MESH:D0082

In [28]:
grounding_map, names, pos_labels = [{'histocompatibility complex molecules': 'MESH:D008285',
  'histocompatibility molecules': 'MESH:D008285',
  'major his tocompatibility complex': 'MESH:D008285',
  'major histo compatibility complex': 'MESH:D008285',
  'major histocom patibility complex': 'MESH:D008285',
  'major histocompability complex': 'MESH:D008285',
  'major histocompati bility complex': 'MESH:D008285',
  'major histocompatibil ity complex': 'MESH:D008285',
  'major histocompatibility': 'MESH:D008285',
  'major histocompatibility antigen': 'MESH:D008285',
  'major histocompatibility antigen complex': 'MESH:D008285',
  'major histocompatibility class': 'MESH:D008285',
  'major histocompatibility com plex': 'MESH:D008285',
  'major histocompatibility complex': 'MESH:D008285',
  'major histocompatibility complex antigen': 'MESH:D008285',
  'major histocompatibility complex class': 'MESH:D008285',
  'major histocompatibility complex proteins': 'MESH:D008285',
  'major histocompatibility ibility complex': 'MESH:D008285',
  'major histocompatibilitycomplex': 'MESH:D008285',
  'major histocompatibilty complex': 'MESH:D008285',
  'major histocompatiblity complex': 'MESH:D008285',
  'mental health care': 'ungrounded',
  'mental health clinical': 'ungrounded',
  'mental health composite': 'ungrounded',
  'mental health conditions': 'ungrounded',
  'mhc': 'ungrounded',
  'moisture holding capacity': 'ungrounded',
  'myosin heavy chain': 'FPLX:MYH',
  'myosin ii heavy chain': 'FPLX:MYH'},
 {'MESH:D008285': 'Major Histocompatibility Complex', 'FPLX:MYH': 'MYH'},
 ['FPLX:MYH', 'MESH:D008285']]
  

In [16]:
excluded_longforms = ['mhc']

In [17]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [19]:
additional_entities = []

In [31]:
unambiguous_agent_texts = {}

In [20]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entities = pos_labels + additional_entities
entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [19]:
intersection1

[('CHEBI:CHEBI:47344', 'CHEBI:CHEBI:47344', 0),
 ('CHEBI:CHEBI:47344', 'CHEBI:CHEBI:5130', 0),
 ('CHEBI:CHEBI:47344', 'HGNC:9600', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D005189', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D005193', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D005454', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D008164', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D014760', 0),
 ('CHEBI:CHEBI:47344', 'NCIT:C32623', 0),
 ('CHEBI:CHEBI:5130', 'CHEBI:CHEBI:47344', 0),
 ('CHEBI:CHEBI:5130', 'CHEBI:CHEBI:5130', 1314),
 ('CHEBI:CHEBI:5130', 'HGNC:9600', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D005189', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D005193', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D005454', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D008164', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D014760', 0),
 ('CHEBI:CHEBI:5130', 'NCIT:C32623', 0),
 ('HGNC:9600', 'CHEBI:CHEBI:47344', 0),
 ('HGNC:9600', 'CHEBI:CHEBI:5130', 0),
 ('HGNC:9600', 'HGNC:9600', 0),
 ('HGNC:9600', 'MESH:D005189', 0),
 ('HGNC:9600', 'MESH:D005193', 0),
 ('HGNC:9600', 'MESH:D005454', 0),
 ('HGN

In [75]:
intersection2

[('CHEBI:CHEBI:27812', 'CHEBI:CHEBI:27812', 0),
 ('CHEBI:CHEBI:27812', 'CHEBI:CHEBI:7035', 0),
 ('CHEBI:CHEBI:27812', 'FPLX:Mechanosensitive_ion_channels', 0),
 ('CHEBI:CHEBI:27812', 'HGNC:7321', 0),
 ('CHEBI:CHEBI:27812', 'MESH:D000080364', 0),
 ('CHEBI:CHEBI:27812', 'MESH:D000604', 0),
 ('CHEBI:CHEBI:27812', 'MESH:D059630', 0),
 ('MESH:D059630', 'CHEBI:CHEBI:27812', 0),
 ('MESH:D059630', 'CHEBI:CHEBI:7035', 0),
 ('MESH:D059630', 'FPLX:Mechanosensitive_ion_channels', 0),
 ('MESH:D059630', 'HGNC:7321', 0),
 ('MESH:D059630', 'MESH:D000080364', 0),
 ('MESH:D059630', 'MESH:D000604', 0),
 ('MESH:D059630', 'MESH:D059630', 2261),
 ('ungrounded', 'CHEBI:CHEBI:27812', 0),
 ('ungrounded', 'CHEBI:CHEBI:7035', 0),
 ('ungrounded', 'FPLX:Mechanosensitive_ion_channels', 0),
 ('ungrounded', 'HGNC:7321', 0),
 ('ungrounded', 'MESH:D000080364', 0),
 ('ungrounded', 'MESH:D000604', 0),
 ('ungrounded', 'MESH:D059630', 0),
 ('MESH:D000604', 'CHEBI:CHEBI:27812', 0),
 ('MESH:D000604', 'CHEBI:CHEBI:7035', 0),


[('HGNC:7321', ['Musculin', 'musculin'])]

In [91]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [21]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-09-24 15:25:01] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-09-24 15:26:01] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.995521795030666 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [22]:
classifier.stats

{'label_distribution': {'MESH:D008285': 3191,
  'FPLX:MYH': 296,
  'ungrounded': 8},
 'f1': {'mean': 0.995522, 'std': 0.003148},
 'precision': {'mean': 0.994842, 'std': 0.003224},
 'recall': {'mean': 0.996271, 'std': 0.003089},
 'FPLX:MYH': {'f1': {'mean': 0.977628, 'std': 0.01858},
  'pr': {'mean': 0.966328, 'std': 0.030069},
  'rc': {'mean': 0.989649, 'std': 0.013655}},
 'MESH:D008285': {'f1': {'mean': 0.997186, 'std': 0.001755},
  'pr': {'mean': 0.99906, 'std': 0.001254},
  'rc': {'mean': 0.995321, 'std': 0.002606}},
 'ungrounded': {'f1': {'mean': 0.466667, 'std': 0.4},
  'pr': {'mean': 0.4, 'std': 0.374166},
  'rc': {'mean': 0.6, 'std': 0.489898}}}

In [23]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [24]:
disamb.dump(model_name, results_path)

In [25]:
print(disamb.info())

Disambiguation model for MHC

Produces the disambiguations:
	MYH*	FPLX:MYH
	Major Histocompatibility Complex*	MESH:D008285

Class level metrics:
--------------------
Grounding                       	Count	F1     
Major Histocompatibility Complex*	3191	0.99719
                             MYH*	 296	0.97763
                      Ungrounded	   8	0.46667

Weighted Metrics:
-----------------
	F1 score:	0.99552
	Precision:	0.99484
	Recall:		0.99627

* Positive labels
See Docstring for explanation



In [26]:
model_to_s3(disamb)